# 爬虫代码实现笔记（二）：建立功能函数模块
因为经常对网页要反复不断地用到笔记一里的代码对网页进行解析和操作，故可把常用的一些代码封装为功能函数，以便在后面使用：

## 九、评论参与人数抽取函数

### 占位符与格式化字符串
因为要重复抽取评论参与人数，那么势必要反复不断地对评论请求不断调用，所以这里要用到格式化字符串以不断地可重复嵌入新闻编号，以保证能反复被调用；

commentsUrl=requests.get('http://comment5.news.sina.com.cn/page/info?version=1&format=json&\
channel=gn&newsid=comos-hfhfwmv1378608&group=undefined&compress=0&ie=utf-8&oe=utf-8&\
page=1&page_size=3&t_size=3&h_size=3&thread=1&callback=jsonp_1531554557168&_=1531554557168')<br>
这是笔记一中请求一条新闻评论信息的Request URL;这条Url字符里的从comos-后至&group前的部份是新闻编号，我们将这部份用{}代替，作为一个占位符，使这个Url字符成为一个格式化字符串：

In [6]:
commentsUrl='http://comment5.news.sina.com.cn/page/info?version=1&format=json&\
channel=gn&newsid=comos-{}&group=undefined&compress=0&ie=utf-8&oe=utf-8&\
page=1&page_size=3&t_size=3&h_size=3&thread=1&callback=jsonp_1531554557168&_=1531554557168'
newsid='hfefkqr0818002'
print(commentsUrl.format(newsid))

http://comment5.news.sina.com.cn/page/info?version=1&format=json&channel=gn&newsid=comos-hfefkqr0818002&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&h_size=3&thread=1&callback=jsonp_1531554557168&_=1531554557168


### 特别说明：不同版块新闻的评论请求是不一样的，主要体现在channel参数上：国内版块channel=gn，而国际版块channel=gj;具体应用时，要视不同版块修改channel参数；

### 建立评论参与人数抽取函数

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import json

def getCommentCounts(newsUrl): #评论参与人数抽取函数
    m=re.search('doc-i(.*).shtml',newsUrl)
    newsid=m.group(1)
    comments=requests.get(commentsUrl.format(newsid))
    jsonStr=comments.text
    #print(jsonStr)
    jsonStr=jsonStr[jsonStr.find('(') + 1:-1]  #截取到有效json字串，从开头到一个左括号后及最后一个右括号前的部份是要的
    jd = json.loads(jsonStr)  #将jsonStr解析出来，并存入jd变量，jd变量为字典型
    return jd['result']['count']['total']  #按照数据的结构层级，取得评论参与总人数

In [7]:
#测试评论参与人数抽取函数
#国内版块,将格式化字符串设为国内模版:channel=gn
commentsUrl='http://comment5.news.sina.com.cn/page/info?version=1&format=json&\
channel=gn&newsid=comos-{}&group=undefined&compress=0&ie=utf-8&oe=utf-8&\
page=1&page_size=3&t_size=3&h_size=3&thread=1&callback=jsonp_1531554557168&_=1531554557168'
newsUrl='http://news.sina.com.cn/c/2018-07-14/doc-ihfhfwmv6168912.shtml'
print(getCommentCounts(newsUrl))

newsUrl='http://news.sina.com.cn/o/2018-06-24/doc-iheirxye9249146.shtml'
print(getCommentCounts(newsUrl))

#国际版块,将格式化字符串设为国际模版:channel=gj
commentsUrl='http://comment5.news.sina.com.cn/page/info?version=1&format=json&\
channel=gj&newsid=comos-{}&group=undefined&compress=0&ie=utf-8&oe=utf-8&\
page=1&page_size=3&t_size=3&h_size=3&thread=1&callback=jsonp_1531554557168&_=1531554557168'

newsUrl='http://news.sina.com.cn/w/sy/2018-07-15/doc-ihfhfwmv6237340.shtml'
print(getCommentCounts(newsUrl))

4231
460
5


## 十、新闻正文信息抽取函数

In [17]:
from datetime import datetime

def getNewsDetail(newsUrl):
    result={}
    res=requests.get(newsUrl)
    res.encoding='utf-8'
    soup=BeautifulSoup(res.text,'html.parser')
    result['title']=soup.select('.main-title')[0].text
    result['newssource']=soup.select('.date-source a')[0].text
    timesource=soup.select('.date-source .date')[0].text
    result['dt']=datetime.strptime(timesource,'%Y年%m月%d日 %H:%M')
    result['article']=' '.join([p.text.strip() for p in soup.select('#article_content p')[:-5]])
    result['editor']=soup.select('.show_author')[0].text.strip().lstrip('责任编辑：')
    result['comments']=getCommentCounts(newsUrl)
    return result    

In [21]:
#测试新闻正文信息抽取函数
#国内版块,将格式化字符串设为国内模版:channel=gn
commentsUrl='http://comment5.news.sina.com.cn/page/info?version=1&format=json&\
channel=gn&newsid=comos-{}&group=undefined&compress=0&ie=utf-8&oe=utf-8&\
page=1&page_size=3&t_size=3&h_size=3&thread=1&callback=jsonp_1531554557168&_=1531554557168'

newsUrl='http://news.sina.com.cn/c/2018-07-14/doc-ihfhfwmv6168912.shtml'
print(getNewsDetail(newsUrl))
#getNewsDetail(newsUrl)

{'title': '环太军演期间 中国情报收集船来到了演习海域', 'article': '原标题：：环太军演期间，中国情报收集船在演习海域行使“航行自由”权 据美国福克斯新闻网报道，一艘隶属于中国海军的情报搜集船一直在美国举办的“环太军演”期间潜伏在美国夏威夷海岸附近进行情报收集工作。 一名美国官员透露称目前还不清楚这艘中国船只的具体位置，但是这名官员表示中国的情报搜集船一直在国际水域航行（夏威夷海岸线12海里之外），并没有影响演习。而五角大楼发言人海军上校克里斯托弗·洛根说：“中国军队最近在南中国海的行动破坏了当地的稳定，违反了在国际水域自由航行的国际准则。”随后又补充说：“中国的行为不符合环太平洋军演的原则和目的。” 参加此次换太平洋军演的智利海军陆战队指挥官Pablo Nieman准将对美国记者表示说：“对于我来说，这次环太平洋军演非常令人失望，这个海域上有一艘不在邀请名单上的船只存在，这很可能会破坏这次演习。我希望并期望所有舰员都采取专业行动，以便我们继续专注于手头的工作，并以合作精神为基础，顺利完成这次演习。“报道中还指出这艘中国海军情报收集船很可能在7月11日（星期三）抵达了美国夏威夷周边水域，目前依然在美国专属经济区内活动。报道称美国海军已经采取“一切防范措施来保护重要的信息不被窃取。” 根据报道，此次环太平洋军演共有26个国家，47艘水面舰艇，5艘潜艇，18艘国家陆军部队，200多架飞机和25，000名人员参与，被媒体誉为“世界最大的海上联合军事演习”。日本“伊势”号直升机驱逐舰、澳大利亚“阿德莱德”号两栖攻击舰、美国航母“卡尔·文森”号、印度尼西亚“孟加锡”号两栖舰等太平洋地区的大型军舰都参与了此次演习。而原本中国海军在今年环太军演受邀之列，不过今年5月美国却取消对中国参加环太平洋国际军演的邀请。现在看来，中国海军显然并没有错过这次演习，并且以另一种形式参与其中。', 'dt': datetime.datetime(2018, 7, 14, 22, 41), 'editor': '张迪', 'newssource': '环球时报', 'comments': 4793}


### 十一、获取新闻列表页中的新闻链接

以新浪国内新闻板块为例：http://news.sina.com.cn/china/
<br>在该页内的新闻列表上向下滚动时，用开发者工具发现新闻列表是动态更新的（异步通讯），所以无法直接在network的doc栏下观察到新闻列表中的新闻标题和链接等相关信息，所以要进一步分析，才能找到新闻列表的相关信息；<br>
一般在开发者工具中查找指定信息的步骤先后是：network:doc->XHR->js;<br>
通过上述步骤，在js中发现了新闻列表信息(如果没有发现，就用鼠标下滚新闻列表，让它产生异步通讯，这样更好观查)：<br>
<img src="images/11.jpg" /><br>
这里说明下：<br>首先：你在发现包含新闻列表的json数据是包含在一个回调函数：newsloadercallback（）中的,所以开发者工具将其看成了js脚本，将其放在了js栏下；<br>其次：观查其Header栏，发现请求URL为：<br>
http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=1&callback=newsloadercallback&_=1531628210695<br>
于是便可以取得新闻列表页的请求链接了，而且要注意这里的page参数，这个参数指明了要请求新闻列表的第几页

In [52]:
res=requests.get('http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=1&callback=newsloadercallback&_=1531630214332')
jd=json.loads(res.text.lstrip('  newsloadercallback(').rstrip(');')) #用lstrip,rstrip方法去掉json数据左右的函数包裹
for ent in jd['result']['data']:
    print(ent['url'])

http://news.sina.com.cn/c/gat/2018-07-15/doc-ihfhfwmv7009976.shtml
http://news.sina.com.cn/o/2018-07-15/doc-ihfhfwmv6877568.shtml
http://news.sina.com.cn/c/2018-07-15/doc-ihfhfwmv6829497.shtml
http://news.sina.com.cn/o/2018-07-15/doc-ihfhfwmv6421688.shtml
http://news.sina.com.cn/o/2018-07-14/doc-ihfhfwmv6392420.shtml
http://news.sina.com.cn/c/nd/2018-07-14/doc-ihfhfwmv6372909.shtml
http://news.sina.com.cn/o/2018-07-14/doc-ihfhfwmv1228059.shtml
http://news.sina.com.cn/c/nd/2018-07-14/doc-ihfhfwmv1093330.shtml
http://news.sina.com.cn/c/2018-07-14/doc-ihfhfwmv1020416.shtml
http://news.sina.com.cn/o/2018-07-14/doc-ihfhfwmv0658752.shtml
http://news.sina.com.cn/o/2018-07-14/doc-ihfhfwmv0619958.shtml
http://news.sina.com.cn/o/2018-07-14/doc-ihfhfwmv0583519.shtml
http://news.sina.com.cn/w/2018-07-14/doc-ihfhfwmv0545109.shtml
http://news.sina.com.cn/c/2018-07-14/doc-ihfhfwmv0544819.shtml
http://news.sina.com.cn/c/nd/2018-07-14/doc-ihfhfwmv0502351.shtml
http://news.sina.com.cn/c/nd/2018-07-13/do

## 十二、抽取指定新闻列表页内新闻链接并获取新闻正文信息函数

上面的代码将新闻列表页中的第一页（page=1）里的新闻链接全部抽取了出来，但是新闻列表页有很多页，如果每一页都要这么写几行代码没显得不值得，不如将上面的代码封装成一个函数，将指定列表页面中的新闻链接抽取出来并同时将这页新闻的正文信息也一并获取出来：

In [53]:
def parseListLinks(url):
    newsdetails=[]
    res=requests.get(url)
    jd=json.loads(res.text.lstrip('  newsloadercallback(').rstrip(');')) #用lstrip,rstrip方法去掉json数据左右的函数包裹
    for ent in jd['result']['data']:
        newsdetails.append(getNewsDetail(ent['url'])) #获取列表页内每条新闻的链接后直接用刚才定义getNewsDetail方法获取
                                                      #每条新闻正文相关信息字典，并将其加入到newsdetails列表中保存
    return newsdetails

In [54]:
#测试parseListLinks函数(下面代码执行时略要等待，因为一下子要抓取二十多条新闻)
url='http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=1&callback=newsloadercallback&_=1531630214332'
print(parseListLinks(url))
print(len(parseListLinks(url))) #这句代码结果应为22，因为一个列表页上有22条新闻链接（url里的参数：show_num=22决定）

[{'title': '台湾小学生称“我来自中国台湾省” 台下掌声雷动', 'article': '原标题：台湾小学生称“我来自中国台湾省” 台下掌声雷动！ 海外网7月15日电 台湾小学生林子咏，在小学三年级时转学到厦门读书，日前在参加大陆电视节目时，感谢同学的陪伴让她适应在中国大陆的生活，之后演唱《童年》来纪念与同学们一起求学的经历。 林子咏在介绍自己时，向台下大声说道：“三年级的时候，我从台湾省，转来厦门念书。”台下同学们掌声雷动，表现出对她的友好和鼓励。 大陆网友纷纷留言点赞，“这个台湾省！很棒了”“台湾省，说的非常好”“给台湾省来的小妹妹点赞”“台湾来的小朋友，要不要戴红领巾”。岛内网友也感慨，“‘日月潭是我国台湾省最大的一个湖。它在台中附近的高山上。那里群山环绕，树木茂盛，周围有许多名胜古迹。’这是每个大陆小学生都会背诵的课文啊！”还有留言直接说出了“九二共识”。 “台湾人是大陆人的同胞” 和林子咏有相同看法的岛内小学生比比皆是，一位名叫程艾葳的台湾女生此前分享过一段视频。视频中，她向台湾介绍大陆小学课本中如何描述台湾。她说，大陆是这么教育孩子的：台湾是祖国的宝岛，台湾人是大陆人的同胞。她表示，“他们（大陆）对台湾非常友善，台湾人不应该乱骂大陆人。” 相比较而言，自蔡英文当局上台以来，一面大讲“善意”，一面大搞“去中国化”，掩盖两岸同文同种的事实，这种“嘴炮”善意不仅虚假，甚至卑劣。台当局的如意算盘是，以所谓“天然独”冲撞两岸关系，但注定不会得逞。 “天然独”遭现实打脸 对于“天然独”说法，洪秀柱就曾直言，台湾青年不是“天然独”，而是受到教育产生的“人造独”。 从蔡英文执政百日起，年轻人对蔡英文支持度的民调数据便一直呈下滑趋势，今年蔡英文执政满两周年，各项民调对她的不满意度都已过半，其中20岁至29岁的年轻人最不满。“天然独”明显是个假命题，是意识形态的捆绑。 新党青年成员，1987年出生的王炳忠此前就表示，不是“天然独”，应是“人工独”，就如食品中有塑化剂，不是天然粮食，但吃久了之后会一点一滴伤身体。（海外网 朱箫）', 'dt': datetime.datetime(2018, 7, 15, 1, 47), 'editor': '霍宇昂', 'newssource': '海外网', 'comments': 6555}, {'title': '博彩公司开

22


## 十三、使用for循环批量抓取多个新闻列表页中的新闻正文信息

### 使用占位符与格式字符串来枚举多个列表页，并获取其中新闻的链接

In [45]:
#将下行url字符中的page参数值设为{},使之成为一个格式字符串
url='http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page={}&callback=newsloadercallback&_=1531630214332'
for i in range(1,10): #i从1迭代至9
    newsurl=url.format(i) #用i值代入格式字符串中的{}位置
    print(newsurl)

http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=1&callback=newsloadercallback&_=1531630214332
http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=2&callback=newsloadercallback&_=1531630214332
http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=3&callback=newsloadercallback&_=1531630214332
http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=4&callback=newsloadercallback&_=1531630214332
http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_al

### 批次抓取多页新闻列表页内的新闻正文信息

In [49]:
#将下行url字符中的page参数值设为{},使之成为一个格式字符串
url='http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page={}&callback=newsloadercallback&_=1531630214332'
news_total=[] #用于保存所有列表页上抓取的新闻正文信息
for i in range(1,3): #抓取的列表页为2页，可自己更改页数范围
    newsurl=url.format(i) #用i值代入格式字符串中的{}位置
    newsary=parseListLinks(newsurl) #抓取指定列表页上的新闻放入newsary这个列表型变量
    news_total.extend(newsary) #将newsary列表并入news_total列表，这样news_total列表中保存了所有列表页上的新闻正文信息了
print(len(news_total))

44


结果为44，正好是两页列表上的新闻数，验证下来没有问题；